### PART1

In [2]:
import numpy as np
import pandas as pd
import os
import zipfile
import requests
import json 
from bs4 import BeautifulSoup 
import boto3 
import time
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [3]:
url = 'https://ies-fsv.s3.eu-central-1.amazonaws.com/studentsSets/45247808.zip'
response = requests.get(url)

with open('data.zip', 'wb') as file:
    file.write(response.content)

In [4]:
zip_file_path = 'data.zip'
with zipfile.ZipFile(zip_file_path, 'r') as z:
    print(z.namelist())

['data/MU.csv', 'data/MCD.csv', 'data/LIN.csv', 'data/DXCM.csv']


In [9]:
os.listdir('.')

['Midterm_solution.ipynb', '.DS_Store', 'data.zip', '.git', 'data']

In [10]:
df_MU=pd.read_csv('data/MU.csv',parse_dates=["Date"], date_format="%d-%m-%y")
df_MCD=pd.read_csv('data/MCD.csv',parse_dates=["Date"], date_format="%d-%m-%y")
df_LIN=pd.read_csv('data/LIN.csv',parse_dates=["Date"], date_format="%d-%m-%y")
df_DXCM=pd.read_csv('data/DXCM.csv',parse_dates=["Date"], date_format="%d-%m-%y")

In [17]:
df_MU['company']='MU'
df_MCD['company']='MCD'
df_LIN['company']='LIN'
df_DXCM['company']='DXCM'

In [18]:
df_MU.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company
0,2017-01-03,21.632,22.289,21.613,22.103,24962900,0.0,0.0,MU
1,2017-01-04,22.220,22.387,21.809,21.917,20426800,0.0,0.0,MU
2,2017-01-05,21.956,22.122,21.524,21.671,19759400,0.0,0.0,MU
3,2017-01-06,21.828,21.838,21.544,21.603,17782200,0.0,0.0,MU
4,2017-01-09,21.505,22.103,21.505,21.897,21537200,0.0,0.0,MU


In [19]:
df_MCD.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company
0,2017-01-03,101.373,101.490,98.961,99.510,5123800,0.0,0.0,MCD
1,2017-01-04,98.994,99.610,98.312,99.393,3780800,0.0,0.0,MCD
2,2017-01-05,99.277,99.992,99.077,99.576,4261100,0.0,0.0,MCD
3,2017-01-06,98.936,101.074,98.595,100.458,4089200,0.0,0.0,MCD
4,2017-01-09,100.458,100.708,100.100,100.184,3021800,0.0,0.0,MCD


In [20]:
df_merged=pd.concat([df_MU,df_MCD,df_LIN,df_DXCM], axis=0, ignore_index=True).sort_values('Date')
df_merged.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company
0,2017-01-03,21.632,22.289,21.613,22.103,24962900,0.0,0.0,MU
3020,2017-01-03,103.284,103.503,101.664,102.347,2364200,0.0,0.0,LIN
4530,2017-01-03,14.875,14.913,14.440,14.562,5717200,0.0,0.0,DXCM
1510,2017-01-03,101.373,101.490,98.961,99.510,5123800,0.0,0.0,MCD
1,2017-01-04,22.220,22.387,21.809,21.917,20426800,0.0,0.0,MU


#### TASK1